### Snow level extraction:

In [61]:
import json
import pandas as pd
from shapely.geometry import shape, Point
import glob
import datetime

We import the accident dataframe and load the topojson files names

In [66]:
# Load accident DataFrame
df_accidents = pd.read_excel('../data/accidents/df_accidents_final.xlsx')
# Obtaine snow topojson files names
file_names = glob.glob("../data/json-maps/*.json")

We define a function that allows for each accident, to select the topojson file:
- If we have topojson file containing the total snow levels ('hstop') for the accident's specific date, we return this file.
- If only the topojson containing fresh snow levels is available ('hn1'), we select the topojson file containing total snow level from 1 to 3 days before the accident, and we add the fresh snow from the accident's date, so that we manage to compute the real snow level.

In [67]:
# Selects the topojson file
def get_file(row):
    
    evol = 0
    match = []
    fresh = []
    file_types = ['hstop','hsr2000','hsr2500','hn1','hn3']
    date_type = row.date
    while (not match and evol <= 3):  
        date = str(date_type.year) + str('%02d' % date_type.month) + str('%02d' % date_type.day)
        matching = [s for s in file_names if date in s]

        for i in range(0,len(file_types)):
            m = [s for s in matching if file_types[i] in s]
            if m != []:
                if i in [3,4] and evol == 0:
                    fresh = m[0]
                else:
                    match = m[0]
                break
        date_type = date_type - datetime.timedelta(days=1)
        evol = evol +1
    return [match, fresh]

# 30 13 1 214 0

Snow levels for each accident are added to the accident dataframe.

In [68]:
no_file = []
# construct point based on lon/lat returned by geocoder
for ind,row in df_accidents.iterrows():
    
    point = Point(row.Longitude, row.Latitude)      
    #file = get_file(row) 
    [file, fresh_file] = get_file(row)
    
    if not file:
        no_file.append(ind)
    else:
        if not fresh_file:
            fresh_snow_level = 0
        else:   
            fresh_snow_data = json.load(open(fresh_file))
            for feature in fresh_snow_data['features']:
                polygon = shape(feature['geometry'])
                if polygon.contains(point):
                    fresh_snow_level = feature['properties']['snow_level']
                    df_accidents.loc[ind,'fresh_snow_level'] = fresh_snow_level
        
        snow_levels_data = json.load(open(file))

        # check each polygon to see if it contains the point        
        for feature in snow_levels_data['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                snow_level = feature['properties']['snow_level']
                df_accidents.loc[ind,'snow_level'] = snow_level
        

# len(df_accidents[df_accidents.snow_level.notnull()]) 252/275

In [69]:
len(df_accidents[df_accidents.snow_level.notnull()]) #/ len(df_accidents)

247

In [70]:
df_accidents.head(10)

,date,Canton,Starting zone,Elevation,Aspect,Activity,Danger level,caught,buried,killed,Latitude,Longitude,temperature,wind,fresh_snow_level,snow_level
0,2002-01-02,BE,Diemtigen,2360,NNW,Backcountry touring,3.0,1,1.0,1,46.546476,7.528077,-8.16,NaN,5-20 cm,50-80 cm
1,2002-01-03,GR,Klosters-Serneus,2400,W,Off-piste skiing,3.0,1,1.0,1,46.855337,9.815028,-4.40,NaN,NaN,50-80 cm
2,2002-01-13,GR,Davos,2240,E,Off-piste skiing,2.0,1,1.0,1,46.836754,9.830075,-1.44,faible,5-20 cm,5-20 cm
3,2002-01-13,BE,Reichenbach im Kandertal,2975,WNW,Backcountry touring,1.0,3,1.0,2,46.522445,7.818004,-5.85,faible,5-20 cm,5-20 cm
4,2002-01-30,UR,Gurtnellen,2910,NNW,Backcountry touring,2.0,1,1.0,1,46.696070,8.686867,-5.46,NaN,5-20 cm,5-20 cm
5,2002-02-02,GR,Tschiertschen,2500,NW,Off-piste skiing,2.0,1,1.0,1,46.773623,9.592680,NaN,NaN,5-20 cm,5-20 cm
6,2002-02-13,BE,Lauterbrunnen,2450,E,Off-piste skiing,2.0,1,1.0,1,46.561333,7.861444,-2.70,NaN,NaN,5-20 cm
7,2002-02-24,VS,Bagnes,2200,N,Off-piste skiing,3.0,1,1.0,1,46.041922,7.200050,NaN,NaN,NaN,80-120 cm
8,2002-02-25,VS,Evolène,2480,SE,Off-piste skiing,3.0,2,1.0,1,46.026699,7.466402,-6.88,faible,NaN,80-120 cm
9,2002-02-28,VS,Martisberg,2070,E,Off-piste skiing,3.0,1,1.0,1,46.399415,8.088644,NaN,modere,NaN,200-300 cm


In [71]:
df_accidents.to_csv('accident.csv')